# 品詞分析

POS-VAEにおいて入力する品詞が
- どれだけの粒度であるべきか
- 処理等必要か
- いらない品詞等あるか

In [1]:
import init

In [2]:
import pandas as pd
df = pd.read_pickle('../data/eccos_v2/df_cosme_parsed.pickle')

In [7]:
# df

## 名詞ごとに集計

In [8]:
len(df.mecab_tagged_text.tolist())

49552

In [44]:
%%time
pos_token_list =[]
for i, tags in enumerate(df.mecab_tagged_text.tolist()):
    for tag in tags:
        pos_token_list.append({'pos': tag['pos_list'], 'origin': tag['origin']})

CPU times: user 490 ms, sys: 56.1 ms, total: 546 ms
Wall time: 544 ms


In [100]:
df_pos = pd.DataFrame(pos_token_list)
df_pos['count'] = 1
df_pos_sample = df_pos.drop_duplicates('pos').set_index('pos')[['origin']]
df_pos_count = df_pos.groupby('pos').count()[['count']]
df_pos_count = pd.merge(df_pos_sample, df_pos_count, left_index=True, right_index=True).sort_values('count', ascending=False)

In [101]:
# df_pos = df_pos.reset_index()
len(set(df_pos[df_pos.pos == '名詞-一般'].origin.tolist()))

7223

In [115]:
%%time
import random
pos_set_count_list = []
for pos in df_pos_count.reset_index().pos.tolist():
    token_list = df_pos[df_pos.pos == pos].origin.tolist()
    token_set = set(token_list)
    pos_set_count_list.append({'pos': pos, 'set_count': len(token_set), 'sample': ', '.join(random.sample(token_set, min(5, len(token_set))))})
df_pos_set_count = pd.DataFrame(pos_set_count_list)

CPU times: user 8.24 s, sys: 0 ns, total: 8.24 s
Wall time: 8.23 s


In [116]:
df_parsed = df_pos_count.reset_index().merge(df_pos_set_count, on='pos', how='outer')

In [118]:
koyuu_noun_key = ['名詞-一般', '名詞-固有名詞-一般', '名詞-固有名詞-人名-一般', '名詞-固有名詞-組織', '名詞-固有名詞-人名-名', 
                  '名詞-固有名詞-地域-一般', '名詞-固有名詞-人名-姓', ]

In [120]:
df_parsed[df_parsed.pos.isin(koyuu_noun_key)]

,pos,origin,count,set_count,sample
0,名詞-一般,肌,148358,7223,"大学生, ジェルティントグロス, 最高, あか抜け, 空き"
2,名詞-固有名詞-一般,2点,68028,4546,"ツッコミ, ポフツカワ, 6色, 91.8%, クンクン"
23,名詞-固有名詞-人名-一般,エスティローダー,5454,571,"小原正子, ヘアー, ライトカラー, 高須英津子, ミュラ"
26,名詞-固有名詞-組織,バリア,3513,741,"ペンシルアイライナー, デル, ウォーターレジスタント, asuoil, ベアスキンファンデーション"
39,名詞-固有名詞-人名-名,一人,1328,222,"種, 諭吉, 泉, 唯, 奈津子"
40,名詞-固有名詞-地域-一般,目覚,1134,142,"池袋, ちの, 医療センター, 表参道, 鹿児島"
42,名詞-固有名詞-人名-姓,クレイ,985,185,"根本, 方, 紫, 泉, 山崎"


In [142]:
# df_pos_uq[df_pos_uq.pos == '名詞-固有名詞-組織'].origin.tolist()

In [143]:
# df_pos_uq = df_pos.drop_duplicates('origin')

In [144]:
# df_noun_ippann = df_pos_uq[df_pos_uq.pos == '名詞-一般']

In [145]:
# df_noun_ippann[df_noun_ippann.origin.str.contains('ファンデーション')]

In [146]:
# df_noun_ippann[df_noun_ippann.origin.str.contains('ライナー')]

In [121]:
# df_parsed[df_parsed.pos.str.contains('名詞')]

In [152]:
from metric import flat_list

In [157]:
flat_list([[1, 2, 3, 4]], 4)

[(1, 2, 3, 4)]